In [1]:
import qick

In [2]:
help(qick)

Help on package qick:

NAME
    qick

PACKAGE CONTENTS
    averager_program
    qick
    qick_asm

FILE
    /usr/local/lib/python3.6/dist-packages/qick/__init__.py




In [3]:
from qick.qick_asm import *

In [4]:
freq2reg(f=100)

69905066

In [5]:
from qick.qick import *

In [6]:
gauss(5,5,100,30)

array([1.10363832e+001, 1.58187727e+001, 2.09302898e+001, 2.55643137e+001,
       2.88236832e+001, 3.00000000e+001, 2.88236832e+001, 2.55643137e+001,
       2.09302898e+001, 1.58187727e+001, 1.10363832e+001, 7.10783276e+000,
       4.22575263e+000, 2.31914221e+000, 1.17491685e+000, 5.49469167e-001,
       2.37211622e-001, 9.45333480e-002, 3.47768752e-002, 1.18100712e-002,
       3.70229412e-003, 1.07138549e-003, 2.86204886e-004, 7.05772560e-005,
       1.60660434e-005, 3.37605524e-006, 6.54887339e-007, 1.17268153e-007,
       1.93842953e-008, 2.95785167e-009, 4.16638316e-010, 5.41749431e-011,
       6.50270665e-012, 7.20520434e-013, 7.36978611e-014, 6.95856849e-015,
       6.06514755e-016, 4.87999386e-017, 3.62454606e-018, 2.48510310e-019,
       1.57286570e-020, 9.18957259e-022, 4.95627535e-023, 2.46758420e-024,
       1.13408335e-025, 4.81143267e-027, 1.88434444e-028, 6.81243877e-030,
       2.27353358e-031, 7.00416835e-033, 1.99190316e-034, 5.22921269e-036,
       1.26724572e-037, 2

In [7]:
from qick.averager_program import *

In [8]:
class LoopbackProgram(AveragerProgram):
    def __init__(self,cfg):
        AveragerProgram.__init__(self,cfg)

    def initialize(self):
        cfg=self.cfg   
        r_freq=self.sreg(cfg["res_ch"], "freq")   #Get frequency register for res_ch
        self.cfg["adc_lengths"]=[self.cfg["readout_length"]]*2          #add length of adc acquisition to config
        self.cfg["adc_freqs"]=[adcfreq(self.cfg["pulse_freq"])]*2   #add frequency of adc ddc to config
        
        if self.cfg["pulse_style"] == "const":
            self.add_pulse(ch=self.cfg["res_ch"], name="measure", style=self.cfg["pulse_style"], length=self.cfg["pulse_length"])  #add a constant pulse to the pulse library
        if self.cfg["pulse_style"] == "arb":
            self.add_pulse(ch=self.cfg["res_ch"], name="measure", style=self.cfg["pulse_style"], idata = self.cfg["idata"]) 
        
        freq=freq2reg(adcfreq(cfg["pulse_freq"]))  # convert frequency to dac frequency (ensuring it is an available adc frequency)
        self.pulse(ch=cfg["res_ch"], name="measure", freq=freq, phase=0, gain=cfg["pulse_gain"], t= 0, play=False) # pre-configure readout pulse
        self.synci(200)  # give processor some time to configure pulses
    
    def body(self):
        self.trigger_adc(adc1=1, adc2=1,adc_trig_offset=self.cfg["adc_trig_offset"])  # trigger the adc acquisition
        if self.cfg["pulse_style"] == "const":
            self.pulse(ch=self.cfg["res_ch"], length=self.cfg["pulse_length"], play=True) # play readout pulse
        if self.cfg["pulse_style"] == "arb":
            self.pulse(ch=self.cfg["res_ch"], play=True) # play readout pulse
        self.sync_all(us2cycles(self.cfg["relax_delay"]))  # sync all channels